In [1]:
## todo fix classes and labels code in detection.py
##tflite tends to give garbage classes to null detections

# INFERENCE - TEST MODE

In [1]:
import numpy as np
from scipy.spatial import distance as dist
import itertools


def get_pixel_scaler(rects,person_width,resolution):
    top_line=[[0.0, 0.4], [1.0, 0.4]]

    bot_line=[[0.0, 0.6], [1.0, 0.6]]
    
    centroid_list=[]


    for (i, (ymin, xmin, ymax, xmax)) in enumerate(rects):
               # use the bounding box coordinates to derive the centroid
                cx = int((xmin + xmax) / 2.0)
                cy = int((ymin + ymax) / 2.0)
                centroids=(cx, cy)
                
                norm_cy=cy/resolution[1]
                
                if norm_cy>=0.4 and norm_cy<=0.6:
                   
                    
                    if xmax-xmin>= ymax-ymin:
                        dist=xmax-xmin
                    else:
                        dist=ymax-ymin
                    
                    person_width.append(dist)    
                    
    
    
    return person_width

def get_box_width(rects,pixel_scale):

    
    centroid_list=[]
    for (i, (ymin, xmin, ymax, xmax)) in enumerate(rects):
        # use the bounding box coordinates to derive the centroid
        
        x_scaled=(xmax-xmin)*pixel_scale
        y_scaled=(ymax-ymin)*pixel_scale
        x_scaled=round(x_scaled)
        y_scaled=round(y_scaled)
        
        cx = int((xmin + xmax) / 2.0)
        cy = int((ymin + ymax) / 2.0)
        
        centroid_list.append([[[xmax,cy],[cx,ymax]],[x_scaled,y_scaled]])
    return centroid_list
               
def compute_distance(image,rects,pixel_scale,thresh=20):
    centroid_list=[]
    
    for (i, (ymin, xmin, ymax, xmax)) in enumerate(rects):
        cx = int((xmin + xmax) / 2.0)
        cy = int((ymin + ymax) / 2.0)
        
        centroid_list.append([cx,cy])
        
    for a,b in list(itertools.combinations(range(len(centroid_list)), 2)):
        ca=centroid_list[a]
        cb=centroid_list[b]
        
        point_dist=dist.euclidean(ca,cb)*pixel_scale
        
        if point_dist<=thresh:
            mid_x=int((ca[0]+cb[0])/2)
            mid_y=int((ca[1]+cb[1])/2)
            cv2.putText(image, 'ALERT',( mid_x,mid_y),cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1)
            cv2.line(image, tuple(ca), tuple(cb),(0,0,255), 1)
        else:
            mid_x=int((ca[0]+cb[0])/2)
            mid_y=int((ca[1]+cb[1])/2)
            cv2.putText(image, '{} inch'.format(int(point_dist)),( mid_x,mid_y),cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255, 255), 1)
        
            cv2.line(image, tuple(ca), tuple(cb),(0,255,255), 1)
        
        
        
    return image
            
def draw_dist(objects,image):
    #cv2.line(image, self.line_points[0],self.line_points[1], (0, 255, 255), 2)

    for centroid,scaler in objects:
        
        
        text = "{} inch".format(int(scaler[0]))
        cv2.putText(image, text, (centroid[0][0], centroid[0][1]+5),cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)
        text = "{} inch".format(int(scaler[1]))
        cv2.putText(image, text, (centroid[1][0]+5 , centroid[1][1]),cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)
        
        #cv2.putText(image, 'dist', (centroid[0] + 10, centroid[1] + 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        

    # loop over the info tuples and draw them on our frame
    #info = [("Up", self.totalUp),("Down", self.totalDown)]
    #for (i, (k, v)) in enumerate(info):
    #    text = "{}: {}".format(k, v)
    #    cv2.putText(image, text, (10, h - ((i * 20) + 20)),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    return image
            

In [2]:
from tracking import Tracking
from detection import Detection
import cv2
import numpy as np

from datetime import datetime

wait_time=[]
list_centroids=[]

def streaming(video_path,resolution=(300,240),color=True,
              model_kwargs={},
              det_kwargs={},
              track_kwargs={},
              track=True,draw=True,
              output_path=None,display=False,new_fps=None):
    
    
    record= True if output_path is not None else False
    dist=True
    #initialize model detection
    det=Detection()
    det.load_model(**model_kwargs)
    
    #if tflite, force resolution to tflite model resolution
    if det.tflite:
        resolution=det.resolution
        
    #initialize model tracking
    if track:
        trk=Tracking(resolution,**track_kwargs)
     
    print('W {}, H {}'.format(resolution[0],resolution[1]))
    
    if record:
        output_dir=os.path.split(output_path)[0]
        os.makedirs(output_dir) if not os.path.exists(output_dir) else None
        output_fps=10
        vid = cv2.VideoWriter(output_path,cv2.VideoWriter_fourcc(*"XVID"), output_fps, resolution, True)
        

    
    if video_path is not None and new_fps is not None:
        cap = cv2.VideoCapture(video_path)
        movie_fps=int(cap.get(int(cv2.CAP_PROP_FPS)))
        cap.release()
        eq_space=np.linspace(1,movie_fps,num=new_fps,dtype='int')
        skip_frame=0
        
    person_width=[]    
    prev_u=0
    prev_d=0
    
    
    for frame,fps in stream_video(video_path,resolution):

            if video_path is not None and new_fps is not None:
                if skip_frame>=movie_fps:
                    skip_frame=0

                skip_frame+=1

                if skip_frame not in eq_space.tolist():
                    continue


            frame,image=det.transform_image(frame,resolution)

            results=det.model_inference(image)

            results=det.filter_boxes(results,**det_kwargs)

            results=det.scale_boxes(results,resolution)

            if dist:
                if len(person_width)<=2:

                    person_width=get_pixel_scaler(results[0],person_width,resolution)

                else:

                    pixel_inch_scale=20/np.mean(person_width)
                    cent=get_box_width(results[0],pixel_inch_scale)
                    #frame=draw_dist(cent,frame)

                    if len(results[0])>1:
                        frame=compute_distance(frame,results[0],pixel_inch_scale)

                    #np.mean(person_width)

            if track:
                #setting zero_sum to false disables the percent_cap counter reset
                zero_sum=True
                percent_cap=2.0
                if not zero_sum:
                    percent_cap=None
                    
                    
                
                
                trk.track_objects(results[0])
                capacity_thresh=1
                last_movement=(datetime.now()-trk.wait_times[-1][0]).total_seconds() if len(trk.wait_times)>0 else 0
               
                enter,exit=trk.count_reset(trk.totalUp,trk.totalDown,capacity_thresh,last_movement,percent_cap)
                trk.totalUp,trk.totalDown=enter,exit

                if trk.on_line_update: 
                    
                    trk.wait_times.append([datetime.now(),trk.totalUp,trk.totalDown])
                    trk.wait_times=trk.wait_times[-50:]
                    
                    fill_perc,fill_rate=trk.get_fill_values(enter,exit,capacity_thresh)
                    
                    wait=trk.get_wait_time(fill_perc,trk.wait_times)
                    
                    print('UP: {} DOWN: {}'.format(enter,exit))
                    print( 'wait_time {}  fill% {}  occup {}/{}'.format(wait,fill_perc,fill_rate,capacity_thresh))
                
                    np.save('db/data.npy', np.array([datetime.now(),trk.totalUp,trk.totalDown]))
                

            if fps is not None:
                fps.stop()
                curr_fps=round(fps.fps(),2)

            if draw:
                frame=det.draw_boxes(frame,results,det.labels)    

                if track:
                    frame=trk.draw_tracking(frame)


            if record:
                vid.write(frame)
            if display:
                cv2.imshow('imgs',frame)            
            if cv2.waitKey(1) == ord('q'):break 

    if record:
        vid.release()
        
        output_split=os.path.splitext(output_path)
        new_output_path='{}_{:.1f}{}'.format(output_split[0],fps.fps(),output_split[-1])
        os.rename(output_path,new_output_path)
        
        
    cv2.destroyAllWindows()

In [1]:
import os
from streamutils import stream_video

#video_path='videos/MW-18Mar-28.avi'

#video_path='videos/kores_door.mp4'
video_path='videos/video-4.mp4'

#video_path='videos/MW-18Mar-17.avi'
#video_path='videos/MW-18Mar-28.avi'
#video_path='videos/MW-18Mar-3.avi'
#video_path='videos/video_hard.avi'

#video_path='videos/filename_16.mp4'

#video_path='videos/MW-18Mar-26.mp4'
#video_path='videos/MW-18Mar-20.avi'
#video_path='videos/MW-18Mar-30.mp4'
#video_path='videos/omni1B.avi'


#streaming params

track=True
draw=True
record=False
display=False
device='fps_test'
#resolution=(300,240)
resolution=(240,180)
color=True

#detection params

#model_path='tf_models/omni_mobnet3_180_D75/detect_qt_16.tflite'

model_path='tf_models/omni_mobnet3_240_v3/detect_qt_16.tflite'
label_path='tf_models/omni_mobnet3_240_v3/labelmap.txt'

model_kwargs={'model_path':model_path,'label_path':label_path}
det_kwargs={'min_conf':.60,'max_boxes':10}


#tracking params
angle=0   

#[x,y][x,y]
line_points=[[0, 0.5], [0.8, 0.9]]


#track_kwargs={'line_points':line_points,'buffer_frames':0,'maxDisappeared':3,'maxDistance':50}
track_kwargs={'line_points':line_points,'buffer_frames':3,'maxDisappeared':5,'maxDistance':60}

#fps
new_fps=None

if __name__ == "__main__":

    output_path=None
    if record:
        if video_path is None:
            videoname='sample'
        else:
            videoname=os.path.splitext(os.path.basename(video_path))[0]+'_'+device
        
        
        folder=os.path.basename(os.path.split(model_path)[0])
        filename= os.path.splitext(os.path.basename(model_path))[0]
        output_path='outputs/{}/{}.avi'.format(videoname,'{}_{}'.format(folder,filename))

    streaming(video_path=video_path,resolution=resolution,color=color,
              model_kwargs=model_kwargs,det_kwargs=det_kwargs,
              track=track,track_kwargs=track_kwargs,
              output_path=output_path,display=display,draw=draw,new_fps=new_fps)
            
            

NameError: name 'streaming' is not defined

In [111]:
arr=[5,6]
arr=[8,4]
arr=[30,4]

room_thresh=20

mins_inactive=120
percent_cap=2.0

reset=False


last_movement= 60#(datetime.now()-wait_time[-1][0]).total_seconds()



def count_reset(enter,exit,thresh,last_movement,minutes_inactive=120,percent_cap=2.0,reset=False):
    
    if reset:
        enter,exit=0,0
    
    
    if mins_inactive and last_movement/60>=minutes_inactive:
        enter,exit=0,0
    
    if percent_cap is not None:
        #if percentage of exits exceeds the acceptable cap then counter is reset
        if (enter-exit)/thresh < (-percent_cap):
            enter,exit=0,0
        #if percentage of entrance exceeds the acceptable cap then counter is capped to the threshold  
        if (enter-exit)/thresh > percent_cap:
            enter=thresh
            exit=0
    return enter,exit

def get_fill_values(enter,exit,thresh):
    
    
    fill_rate= max(0,enter-exit)
    fill_perc=fill_rate/thresh

    
    return fill_perc,fill_rate


def get_wait_time(fill_perc,wait_times,max_wait=3600):
    wait_times=[i[0] for i in wait_times]
    time_diff=np.diff(wait_times)
    wait_time=np.median(np.vectorize(lambda x:x.total_seconds())(time_diff))
    
    wait_time= 0 if fill_perc<1 else wait_time
    
    return wait_time

    
enter,exit=count_reset(arr[0],arr[1],room_thresh,last_movement,mins_inactive,percent_cap,reset=reset)
fill_perc,fill_rate=get_fill_values(enter,exit,room_thresh)
wait=get_wait_time(fill_perc,wait_time)

print( 'wait_time {}  fill% {}  occup {}/{}'.format(wait,fill_perc,fill_rate,room_thresh))

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

['video', 'image', 'model', 'detection', 'tracking', 'location', 'counter', 'db', 'credentials', 'other']


{'record': 'True', 'draw': 'True', 'display': 'False', 'test': 'False'}